> "Machines will be capable, within twenty years, of doing any work a man can do" - Herbert Simon, AI pioneer, in 1965

# Introduction

Large language models (LLMs), in particular those classified as generative artificial intelligence (gen AI), increasingly support use cases in finance and economics (@korinek2023gen), including in central banks (@araujo2024artificial). These models are tested for their ability to reason, often boasting seemingly incredible results: for example, OpenAI's GPT-4 boats human-like performance across tests of reasoning and more than 80% correct results in academic and professional micro- and macroeconomics tests (@achiam2023gpt). Still, even such advanced models can fail miserably when it comes to reasoning: for example, an advanced model can correctly solve a logical puzzle requiring reasoning about higher order knowledge, only to fail when irrelevant details are changed (@perez2024testing). Building on results such as this, this work discusses how to systematically measure *economic* reasoning, combining literatures on economic thought and on computer science about gen AI benchmarking. In practical terms, the task at hand is to come up with a benchmark task for economic reasoning, a testing mechanism to measures in a comparable way the level of economic reasoning of an artificial intelligence (AI) model. This task is of first-order importance given the break-neck speed of evolution of LLMs (@yang2023harnessing) and their potential risks (@danielsson2022artificial).

Such benchmark tasks are crucial for the comparison of the abilities of AI models over time and in the cross-section. Results of benchmark tasks are now a staple of the evaluation of LLMs by developers when releasing a model, highlighting its evolution compared to previous versions and to the main peers. Third-party organisations also compile leaderboards with running results that allow the general public to keep track of the most performant models.[^leaderboards] Benchmark tasks are useful because that they provide a comparable metric on which to track the state-of-the-art for the particular abilities that each task measures. Usually, this metric is the percentage of correct answers. However, even if specific tasks have evolved over time to become more challenging, a key challenge is to separate correct answers due to probabilistic association or "stochastic parroting" (@bender2021dangers) from those that are the result of a reasoning process.

[^leaderboards]: Commonly followed leaderboards include LMSYS's ChatbotArena and Huggingface's Open LLM Leaderboard.

This paper proposes a working model of reasoning that can underlie an empirical benchmark task: when confronted with a prompt $Q$,[^prompt] an AI is said to *reason* correctly if it responds with an answer $\alpha$ that simultaneously (a) interprets the prompt, identifying the relevant information for the task and filtering away everything else by ignoring or abstract away irrelevant details, (b) associates $Q$ with any relevant existing commonsense knowledge $\theta$ to answer the question, and (c) applies logical relations such as deduction and induction to $Q$ and $\theta$ to arrive at the correct answer. Formally, each answer is defined as a non-parametric function of the following steps: information filtering $\phi = f(Q)$, knowledge curation $\kappa = k(\phi, \theta)$ and logic attribution $\lambda = l(\kappa, Q)$, $\alpha = A(\lambda, Q)$, where $A$ is the function that returns the correct answer from the prompt $Q$. Each of those three steps above are sequential, and depend on the successful completion of the previous step. The goal is for this model to be simple and intuitive. Throughout the paper, I assume AIs respond to their best ability, meaning that they would reason instead of probabilistically choosing an answer $\tilde{a} = \arg \max_{a} L(a | Q)$.

[^prompt]: Usually this will be a string of text, but more recent models can take multi-model content, ie a combination of text, images, videos, sounds, etc.

The empirical version of this prompt-answering model is $\hat{\alpha}(q) = \hat{\phi} \, \hat{\kappa}|(\phi=\hat{\phi}) \, \hat{\lambda}|(\phi=\hat{\phi}, \kappa=\hat{\kappa})$, where $\phi = \mathbf{1} \prod_{i}^{M_f} f(q + \epsilon_i^f)$ and similarly for $\kappa$ with $k$ and $\lambda$ with $l$, $M_f$, $M_k$ and $M_l$ are the number of variations $\epsilon$ introduced in the seed question $q$ that seek to identify the model's interpretation, knowledge association and logic attribution respectively; the hat denomination points to empirically estimated versions. This model is estimated by assessing answers from the same AI model to multiple versions of seed questions $q \in \mathbf{Q}$, and $\hat{\alpha}$ is only considered to be correct when all of the relevant variations for the same question are answered correctly - in other words, the AI model has evaded "banana skins" that try to trick it into revealing lack of information filtering, spurious knowledge association or faulty logics. The key idea is to leverage insights from the social economics literature and create identifying variation in the questions $q$ presented to AI models, adapted from how this is done with human subjects (@stantcheva2023run). The benchmark result is then $R = M_{\mathbf{Q}}^{-1}\sum_{q \in \mathbf{Q}} \hat{\alpha}(q)$, where $M_{\mathbf{Q}}$ is the number of different seed questions. By a similar token, each of the three steps can be measured separately, building on their empirical identifications $R_{j \in (f, k, l)} = M_{j}^{-1}\sum_{q \in \mathbf{Q}} \hat{j}(q)$. Note that identification of $\kappa$ and $\lambda$ need the sequential conditionality on the previous steps $\phi$ and $\phi, \kappa$ respectively in order to be identified.

This model can be used as a general abstraction for an AI reasoning ability, but two practical adaptations in $Q$ can make it a model for economic reasoning more specifically. First, the scope of topics that are included in $Q$ should ideally focus on issues of significance to economics. At its most essential form, testing for economic reasoning is the same as probing if the model is able to think in terms of logical operators on information that is of relevance to economics. However, this is subjective because economic thought constantly evolves. At the same time, including only "classical" economics carries a high risk of using material that is containd in the training set of AI models. Second, even for each given topic, the types of questions considered relevant in economics are specific. Both of these issues are dealt with in practice by using recent published academic work as source material to construct seed questions.[^other] These sources contain content whose topic and research questions are by definition of interest to the economics field, and moreover have the advantage of being novel by design, creating a natural check for the ability of AI models to generalise reasoning.

[^other]: Similar adaptations could be pursued for other fields.

While evidence amounts that large language models (LLMs) cannot perform advanced reasoning, at least not when fine-tuned or with access to external reference material or sophisticated math funcionalities, it is important to have a challenging reasoning benchmark because the latter two possibilities are increasingly being used. Both fine-tuning on specific data and plugging LLMs into sources of knowlege (as in RAGs) or with plugins as Wolfram Alpha or Mathematica might provide them with this ability and thus it is important to have a robust way of measuring the reasoning abilities of these models. Another argument is that if their current dismal performance is due to lack of data, including being limited to written language only, current and future models will be able to leverage a significant part of the other non-text data (eg, video, audio, pictures) and thus could reasonably attempt to achieve better reasoning.

Similar to many other social disciplines, economics requires the analytical judgment referred to by @robbins1932essay in the analyses of events as a basis to extrapolate and predict, and this has a bearing on how economic reasoning should be benchmarked. Economic inference depends primarily on articulating unobservable quantities, theorecised and estimated on the basis of observable measures. This is unlike other major disciplines. For example, in human and veterinary medicine, all physiological and pathological variables of clinical importance are observable, even if that is not yet technologically feasible today. In the medical sciences, theoretical models merely fill in the gaps in the absence of a technologically feasible complete measurement. In contrast, many economically relevant quantities are latent variables that cannot by definition be observed, and always require a model applied to data to be estimated, implicit or not.

A quantitative test for economic reasoning must take this into account: selecting a correct answer in an economics question through reasoning will always depend on an unobserved transformation of the information received and the existing knowledge. This is important. LMs may also happen to choose the correct answer from either luck of through simple token probability. It is easy to see why a correct answer selected by chance is not informative about the reasoning abilities of a model. The second case requires more explanation: mathematically, LMs are trained to identify the most likely token $\theta$ in a vocabulary $V$ given the tokens in its prompt. In practice, the function is inexcrutable so it is also considered an unobservable transformation. But a few characteristics allows us to distinguish reasoning from prediction. First, reasoning is robust to minutiae and other irrelevant detail. Mathematically, it would be analogous to applying a manifold transformation that retains only the relevant information in a prompt and then applies logic operations on top of them, and on them only. Second, reasoning is locally complete, meaning that an LM that can correctly deduce that A implies B also is able to understand that A' does not imply B, or that A does not imply B'. In other words, a reasoning that appears to be correct but whose obvious corolary is not achieved by an LM cannot be said to have been reasoned in the first place.

Knowledge: linguistic, common and commonsense.

Interpretation. information theory. Shannon.

The main intuition of this work is to combine a number of building blocks of evaluation. 

- the benchmark must be challenging for machines: I use an adjusted version of adversarial filtering (@zellers2019hellaswag) to create answer candidates that are hard for LMs to guess

- the test must incorporate slow-moving evolutions in academic economic thought: evolving test set based on newly published academic work. 

- results related to reasoning must be distinguished as best as possible from the ability to interpret the prompt or from knowledge (implicit or explicit) about economics, ie reasoning is a separate step: sets of perturbations in the spirit of @alzahrani2024benchmarks for each initial task.

the benchmark counts with a mathematical adjustment that takes into account performance across perturbations, penalising results that vary with ....

This benchmark evaluation also addresses a poignant issue for the economics profession: the lack of publicly available data about how these benchmarks are created and any, and tested. For example, @achiam2023gpt are not clear about the academic and profession tests on micro- and macroeconomics that are used amongst various other tests to measure GPT-4's performance in those fields. Conversely, various other benchmark tasks do have a publicly available methodology and even evaluation interface, which greatly facilitates the engagement with model developers, general users and third-party model evaluators.

A major inspiration in the design of the questions and how they can generate identifying variations is the social economics literature. A key reference is @stantcheva2023run. The idea here is that the design of the questionnaire itself can elicit responses that allow for insight into non-observable traits such as reasoning. Many of the insights of this literature carry over naturally to the machine space.[^untested]

[^untested]: Actually testing whether LMs *do not* parrot or "organically" exhibit biases or other behaviours that are assumed to be exclusively human would be an interesting line of research.

## Literature

This work builds on, and seeks to expand, three general literature streams. More technical aspects of this work are based on specific literatures that are discussed within each section.

The first body of works is on benchmarking tasks for AI models. A substantial body of work creates and discusses model benchmarking in general; a voluminous and well-organised compilation of references is @storks2020recent.

Secondly, this paper draws from insighs in the more general AI reasoning literature. As other parts of AI development, it is informed and inspired by neuroscience as well (@HASSABIS2017245). An early and influential contribution is the Chinese room experiment by @searle1980minds and its resulting arguments that AI could not reason by itself. The influential works of @bubeck2023sparks and @wei2022emergent hint at acquisition of advanced capabilities by large-scale language models such as GPT-4, although @bubeck2023sparks also point to many instances where reasoning breaks. @schaeffer2024emergent present evidence that these "emerging abilities" that come with scale are actually a spurious by-product of the choice of metrics to emasure these abilities. @mitchell2023debate summarises the disagreement in the AI academic and practitioner fields as to whether AI models have some form of understanding, and by implication, potentially also reasoning. @wei2022chain claim that writing the prompt in a way that offers chain-of-thought examples improves the reasoning abilities of LLMs, although this was later demonstrated to be as generalisable (@dziri2024faith, @prystawski2024think). @BrowningLeCun2022 argue that AI models trained on written language alone will never be able to reason.

A nascent literature on the evaluation of language models in economic settings. An early foray into questions related to AI's ability to conduct economic reasoning is due to @parkes2015economic. But their angle is more on how AIs can be used to estimate synthetic economic agents - machina oeconomicus - ideal versions of purely rational agents, rather than on the measurement and the implications of AIs acquiring economic reasoning abilities. In any case, @parkes2015economic see economic reasoning as the ability to understand and solve complex game-theoretical environments (eg, the poker example). @mei2024turing do an extensive comparison of personality traits from the behaviour of ChatGPT with human behaviour in games that require cooperation, finding that its performance is consistent with humans, and when it deviates the AI models tend to behave in the altruistic and cooperative than the mass distribution of humans. Interestingly, ChatGPT responds differently to different formulations of the same situation. In contrast to @mei2024turing, this paper and its empirical counterpart are more generall, and discuss reasoning as a whole. Another contrast to that paper is that the current benchmark is focused on reasoning ability only, not personality. @perez2024testing illustrate the brittleness of a leading AI's reasoning, which has markedly lower performance when trivial details in the prompts are different. Similarly, @korinek2023gen report (in his Chat 23) that results from a technical prompt in economics are reasonable but also brittle, with answers changing when prompt wording changes or even simply if the tasks are re-ordered.

# A model of economic reasoning

The result from existing benchmarks is largely, if not completely, directly related to the number of questions correctly answered. However, this measures only the model's ability to answer correctly, *not necessarily* its reasoning capabilities. The latter are part of a latent state space sitting between the input prompt and the answer. More concretely, for an input prompt $X$, which includes a question and any necessary explicit information, the language model is a function $\mathbf{M}$ that maps it to a given response: $\mathbf{M} : X \to y$. In order to show that it is done by reasoning, we need tests (and more specifically, measurements) that convey some information about the inner workings of this function. 

## Reasoning as an abstract of the input

- Input prompt $X$

- Transformed into $g(X, \kappa)$, a state space function that also takes the existing knowledge $\kappa$ and associates it with the prompt to maps it to its abstract fundamentals (similar to manifold learning)

- Result based on $g(X)$.

## A (very) simple model

This section builds on the intuition that in true reasoning, the result should be robust to minute perturbations, ie the model is a constant function over the domain of the input. Formally, both $\mathbf{M}(X) = y$ and $\mathbf{M}(X + \epsilon) = y$ for an infinitesimal $\epsilon$. This implies the derivative with respect to the input prompt is zero. Using as an approachable example the simplest possible neural network, the logistic regression $\mathbf{N}(x) = \sigma(Wx + b)$, such robustness further implies that $\frac{d\mathbf{N}}{d x} = \sigma(Wx + b)(1-\sigma(Wx + b))W = 0$. Because $W$ cannot be a zero vector in a functioning network that is responsive to its inputs and $\sigma(Wx + b)(1-\sigma(Wx + b)) = 0$ has no solution because neither term is 0 or 1 in a sigmoid function with finite inputs, the neural network cannot be a constant function. This extremely simplified example, which holds for recursive architectures of similarly simple layers, does not bode well for the robustness of results given small perturbations in the input prompt.



# Reasoning benchmarks in other fields

- Math

- Medical

- Biology

- Economics

In economics, reasoning will always depend on unobservable thought experiments. Even if all existing data was observable, economic research would still revolve around ideas, counterfactuals and thought experiments. A key idea is Haavelmo's distinction between correlation (which is observed from data) and causation (which is not, and requires a thought experiment) (@heckman2015causal). Similarly, the concept of statistical conditioning (again, which can be observed or estimated from data) and the "fix" operator (which also relies on a model) makes a complete difference.

# A model of reasoning

This section develops a model of reasoning that fits naturally into both natural and artificial LMs. It will serve as the basis for the subsequent analyses and empirical creation of a reasoning benchmark.

Let a sentence $\mathbf{S} = (\theta_1, \theta_2, \theta_3, ...)$ be a sequence of token-location tuples $\theta_x = (\tau, x)$, with each $\tau \in \mathbf{V}$ belonging to a vocabulary $\mathbf{V}$ and $x \in \mathbb{N}^{d_{\text{model}}}$.[^location] Create a function $\pi_{i, C} : \theta, \mathbf{S} \to \{-1, 0, 1\}$ that maps each token into one of three possibilities: the token's information can be considered a adversarial (-1), irrelevant (0) or relevant (1) with respect to the likelihood of individual (or LM) $i$ uttering another sentence C. For example, take the following quote from the character Barf in the 1987 movie Spaceballs, organised as two sentences "I'm a mog. Half man, half dog." and "I'm my own best friend." With word-level tokenisation, $\mathbf{S} = \{("\text{I'm}", 1), ("\text{a}", 2), ("\text{mog}", 3), ("\text{.}", 4), ("\text{Half}", 5), ("\text{man}", 6), ("\text{,}", 7), ("\text{half}", 8), ("\text{dog}", 9), ("\text{.}", 10)\}$ and $\mathbf{C}$ is similarly broken down. This example illustrates that even when there is not a logical connection grounded in truth, tokens in one sentence - even those made up like "mog", can have a bearing on the likelihood of tokens appearing in another sentence. This likelihood can differ depending on the location of the token, which also allows for situations where repeteating of a word $\tau$ is meant to convey different meaning. Another feature of this example is that all $\pi_{\text{Barf}, C}(\theta) = 1 \forall \theta \in \mathbf{S}$. In the alternative sentence "I'm a mog. Half man, half dog. I am alive.", the new component is obviously irrelevant for $\mathbf{C}$: $\prod_{x \in [10, 14]} \pi_{\text{Barf}, C}(\theta_x) = 0$.

[^location]: The location is important because it helps define meaning, along with the actual letter (more generally, symbol) content of th token. Note that in this paper, white spaces are abstracted away for expositional simplicity.

This exposition is important to delve into the reasoning aspect, entirely organised by function $\pi$. Since $\pi_{i, C}$ measures how informative a token is for individual $i$'s $\mathbf{C}$, it constitutes the first aspect of reasoning: to recognise when a token is adversarial, irrelevant or relevant. This step is necessary before the application of any logical rules $\mathcal{l} \in \mathcal{L}$ on the weighted token, $\pi_{i, C}(\theta_x) \theta_x$. The exact underpinnings of these logical rules are beyond the scope of this work - it can be approximated by a possibly non-linear function, $g$. What suffices in this work is to say that reasoning *depends* on correctly classifying the tokens: all relevant tokens must be so identified, lest they be either ignored as the irrelevant ones or taken with the opposite meaning. Similarly, if all relevant tokens are indeed diagnosed correctly but other tokens are also diagnosed as relevant when they are not, then this will cause problems for the correct reasoning. In other words, a first precondition for reasoning is to have a low categorical cross-entropy loss. Intuitively, a pre-condition of reasoning is to correctly interpret the inputs.

Use Taylor expansion on model since its derivative to perturbation should be zero. This gives us a head start in the Taylor expansion. Try to link the T-expanded equation to an estimating equation.

But what determines $\pi_{i, C}$? A combination of knowledges and logical relationships.

Knowledges: linguistic knowledge, common knowledge and commonsense knowledge

Rationales: reasoning from logic

Armed with the sentence-level categorical cross-entropy, the individual can establish chains of thought that will finally lead to reasoning. Again, for simplicity, the exact function is not discussed here, other than that it is a potentially simple or complex way to interact. What is important is to add the categorical cross-entropy to the estimation equation.

**Benchmark testing mechanism**...

# A structural model of reasoning

## Information filtering

Perception should be robust to irrelevant input.

Efficient coding hypothesis (@barlow1961possible, @olshausen1996natural, @loh2014efficient).

Redudancy reduction (@barlow1961possible): "sensory relays recode sensory messages so that their redundancy is reduced but comparatively little information is lost". Based on Shannon's (@shannon1948mathematical) information theory.

The AI literature has of course known this of years, and it inspired the concept of attention (@larochelle2010learning, @mnih2014recurrent), which later inspired the self-attention and ultimately the game-changing transformer architecture (@vaswani2023attention).

Appropriate perception should understand that the information of relevance to understanding a problem is actually much lower-dimensional. In the machine learning literature, this is referred to as the manifold hypothesis. @cayton2008algorithms offers an early review of the main algorithms for estimating empirically the underlying manifold.

@bengio2013representation discusses extensively the idea of representation learning (and its various techniques, mostly unsupervised), which can be seen as manifold learning. However, they might also approximate the wrong manifold or not have a single solution to a same manifold (@lee2023geometric).

## Knowledge association

Assuming a prompt has been correctly parsed, the reasoning mechanism must now match it with the relevant knowledge, which can come from the prompt itself or from commonsense knowledge.

Concept of knowledge association is related to cognition.

Knowledge can be linguistic, common or commonsense (@davis2015commonsense). @mahowald2023dissociating uses insights including from neuroscience to distinguish the first type of knowledge with the latter two (grouped as "functional" knowledge), and argue that LLMs have essentially mastered the former while still having a spotty record on the latter. For example, LLMs learn grammar, semantic, hierarchical structures, abstractions and constructions that provide a realistic linguistic knowledge.

@BRANSFORD1972717 show in experiments that contextual knowledge (in this case akin to commonsense) are essential for proper understanding in humans. The first experiments tested understanding by subjects of a grammatically correct, non-metaphorical passage that required an unusual and very specific, but highly relatable image as context for proper understanding. Note that these characteristics of the passage (correct, non-metaphorical) and of the context (unusual but relatable and easy to understand) both contribute to isolate the identification of this exercise in the aspect of whether contextual knowledge is required. [^prior]

[^prior]: Interestingly, the same paper also demonstrates that prior knowledge itself is not necessarily readily available but needs to be "activated". This is not further discussed in the context of this paper as it is not a mechanism necessary for measuring reasoning abilities in AI models.

Which type of knowledge to match to the prompt? One way of seeing this is through the lends of a query in a knowledge graph. @kleinberg1999authoritative distinguishes specific and broad queries, each giving rise to one problem: that of a scarcity of correct answers and abundance of correct answers, respectively. Further, @kleinberg1999authoritative offers the fundamental ideas of *authority*. Similarly, @kleinberg1999authoritative acknowledges that measuring the authority level of a node in a knowledge graph from explicit information alone (what he calls *endogenous* measure). On the contrary, even so much as using strings from the query itself might mislead answers due to an abundance of other sources that are based on the string and a scarcity of correctly authoritative sources that use the string. Interestingly, while the principal eigenvector of the square of the adjacency matrix offers the weights of authoritativeness especially for broad queries, the non-principal eigenvectors can offer insights into the authoritativeness of more specialised queries, and also due to their negative entries offer authorities of different perspectives (ie, weighting pros and cons).

So the lower-rank approximation of the knowledge graph should vary with how broad/specific a query is, and also with the level of pros/cons required.

In @kleinberg1999authoritative, the authoritativeness measures comes from the eigenvectors of $A^TA$, with the principal eigenvector being the used as a broad authoritativeness metric, and the $n$th eigenvectors for $n>1$ as the more specific, and potentially discordand, authorities.

## Step 3: logic attribution

Attributing the inducive, deducive, etc logic steps to different statements.

@dziri2024faith create a model of task composition.

## Adaptation for economic reasoning

...

## The importance of manifold for reasoning

The first step, interpreting the received impulses (ie, the prompts), involve correctly judging what is relevant and what is not relevant. This is similar for example to how the brain receives an incredible amount of sensory inputs but chooses to focus only on those that are more relevant instead of being overwhelmed with everything else, an observation that has inspired dimensionality-reduction algorithms (eg, isometric mapping, or IsoMap, by @tenenbaum2000global describes how to find global optima while also defining the (much lower) degrees of freedom in a high-dimensional input).

For example, @intrinsic2021 study the intrinsic underlying dimensionality of the manifold of image datasets and find them to be significantly lower. In practice, inputs can even be said to be *union of manifolds* (as verified by @brown2022verifying with image datasets in an exercise similar to the one by @intrinsic2021), which means that each manifold has its own intrinsic dimensionality that is not forced upon the other manifolds. This perspective affords flexibility in the interpretation of identifying variations because they don't necessarily need to probe the same dimensions at each task.

In econometrics, @andrews2016geometric.

The intrinsic dimensionality is modelled mathematically after @kim2019minimax (and adjusted by @levina2004maximum) as...

The estimator by @levina2004maximum plays a big role in the model described here. We use the estimator equation in @intrinsic2021 to inspire the structural equation.

## Insights from human reasoning in economics

Social economics literature.

Behavioural economics literature: @gennaioli2010comes show that people focus on the features that are closer to the data (review this description). Also insights from Thiking Fast and Slow (CITE, reviewed in @shleifer2012psychologists)

## Reasoning iself as a manifold

Since proper reasoning needs to be insensitive to unimportant details, and the vector of changes depends on logical relationships between components, the set of all "reasonable" constructions is not obtained at random but reflects this lower-dimensional, underlying structure, similar to how random pixels would only rarely form human faces.

@gilboa2014analogies argues why economic reasoning works in the way of creating simple, positively wrong but conceptually useful representations of reality, even when economics is studying particular cases. A marked characteristic of such models is their preference for simplicity, a theme also explored by @GILBOA20101757, who study the matching of economic theories to empirical data, generalising the evaluation of how reasonable a theory is through a combination of their likelihood (or goodness-of-fit) with a penalising factor for their complexity. Intuitively, this simplicity in reasoning is suggestive of the manifold hypothesis in reasoning as well.

@rationality2023gilboa sees rationality, or reasoning, also as a robustness to trivial detail, and also discuss different types of reasoning (subjective reasoning, etc). 

A related but not exactly the same perspective is offered by the possibility to identify models partially using random sets, ie abstracting away from point identification to situations where the data is incomplete or is described as an interval (@BERESTEANU201217). In other words, "available data combined with credible maintained assumptions may yield much information about a parameter of interest, even if they do not reveal it exactly." (@MOLINARI2020355), a key insight is to illuminate how *available* data can inform the estimation of models.

# Reasoning about economics

The model above allows us to estimate reasoning while also breaking down some of its components to better understand them. For example, we can estimate any errors in reasoning into an issue with **information filtering**, **knowledge association** and **logic attribution**. The empirical estimation follows.

@theory2022gilboa distinguish between three types of inquity in economic theory: economics itself (analysis of economic phenomena), development of economic methods (the development of analytical tools needed to study economic phenomena) and the methodology of economics (the research/scientific endeavour in economics, including but not limited to theory).[^economics]

[^economics]: In fact, @theory2022gilboa even allude to the blurred lines between economics and the philosophy or sociology of economics. I don't go ino these differences here.

Another insight into *economic* thinking is from the thought experiments first introduced by Marshall (1890) - the ceteris paribus idea - and then later Ragnar Frisch and Trygve Haavelmo, more recently elaborated in more detail and more generally by @heckman2015causal, including the important distinction between correlation and causation. @marschak1944random start their influential paper by acknowledging that economists can't conduct experiments (although that has been relaxed somewhat, it still remains the case at least in macroeconomics).

Reasoning in economics as exploring the latest space through models, or thought experiments, insight generalised by @heckman2023econometric.

@bergemann2022counterfactuals describe how counterfactual predictions can be made in settings where agents behave strategically and both relevant information and the distribution of states of the world (relevant to pay-offs) are unknown. The latent information structure is infinite dimensional.

# Empirical estimation

Each *task* $\theta \in \Theta$ can be asked in various different ways, each one being called a *question* $q \in \theta$. Questions vary with respect to their adversarial aspect; it is this variation within each question that allows the empirical estimation of the effects associated with interpretation or with knowledge. Most of the variations are originally those tested in @alzahrani2024benchmarks. The variation in response between the questions within each task will comprise the evaluation of the actual reasoning capabilities. As alluded to before, the variations are organised into those that measure the stability of a response to adversarial interpretation answers, and those that measure the stability across the knowledge dimension. In practice, each task has hundreds of different $q$. These groups are described in more detail next.

## Variations related to interpretation

There are several classes of variations that can help test an LMs' interpretation. 

### Choice variations

Here the choices remain the same for a task but vary in their order across questions

- random choice order

- biased choice order

- uncommon answer choice symbols

- common but unordered answer choice symbols


### Word variations

The main idea here is to introduce or change words that are irrelevant. This is along the lines of the test conducted by @perez2024testing.

Another one is to conduct random word repetition as if it were a typo

## Variations related to knowledge

Changing key words related to field knowledge with other field knowledge words but that would not make a sense to an expert. This can be compared with just changing the same words into another generic word. Comparing responses between both should indicate the level of knowledge used by the model (should it? need to think more)

One way to test knowledge is to conduct the flip-flop experiment: simply asking LLMs to confirm their answers often make them switch answers, even if their original response was correct (@laban2023you, @xie2023ask). The key idea of these tests is to see if the response changes in the absence of any other changes to the knowledge base (neither on the original model weights obviously but also including the information content of the prompt).

## Estimation formula

The main formula is akin to the linear probability model since $a_{q}$ is either zero or one:

$$
a_{q} = \beta_{\theta} \theta + \beta_{\text{Interpretation}} \eta_q + \beta_{\text{Knowledge}} \kappa_q + \epsilon_q
$$

Another idea to explore is whether these variations can actually instrument interpretation and knowledge. This would allow the formula to estimate the reasoning bit.

# Practical considerations

## Avoiding spillover into training data

The strategy to use newly published academic papers as sources might broadly avoid that most of the content has been used in AI model training. However, most published papers in economics are previously published as working papers, which means they are potentially in the public domain at training time so cannot be guaranteed to be completely novel. While this is mitigated by the arguably low dissemination of secondary material about working papers (for example, one could reasonably conjecture that few recent working papers immediately become the topic of teaching notes or are referred to in more detail by other papers), a more robust practical strategy is needed, especially as the training dataset of many of the most advanced models is not publicly known.

One way of dealing with this is by introducing in a variation of the questions a random string that is almost guaranteed to be unique and that is not found in common text datasets used to train LLMs. This is of course not perfect, because it cannot guarantee that the original paper is not part of training data, but can at least ensure that if the seed questions themselves are for some reason used to train models, this could be identified by model developers (and if the training data is available, also by third-party evaluators).

## Lessons from human surveys

I use a considerable amount of specific advice on human surveys from @stantcheva2023run to generate identifying variation in the questions. Specifically, all the questions avoid jargon to the best extent possible, and only include questions that are either of the coeteris paribus type, or that include as options assessments on the statements of the form "correct", "incorrect", "equal" or "I don't know".[^beyond] Particular care is taken with respect to introducing variations in the seed questions that can help measure each of the three reasoning components of information filtering, knowledge association and logic attribution.

Consideration is given to whether each question should be presented to a separate instance of the LM, or the full questionnaire could be shared in the same "chat", which would be akin to the "few-shot" prompting. Another practical advice as part of the estimation is to prototype the questions (I used GPT-4 for the prototyping).

[^beyond]: Future versions of this benchmark could also include open ended questions (as in @ferrario2022eliciting), and even follow-up questions ("are thre any other reasons"). These open-ended questions that are similar in nature to closed-end questions could be assessed by a fine-tuned LLM.

## Desirable characteristics of a benchmark

A benchmark task for economic reasoning should ideally have the following characteristics in order to be useful and maintain relevance even in a scenario where model developers are able to acquire a significant body of economically relevant texts (eg, new papers).

**Inform performance on different components of reasoning**: An ideal benchmark can help practitioners intuitively grasp the performance of the models in each major "task" that is performed in the process of reasoning. This would help developers and users better understand what the models are good or bad at, and judge their adequateness accordingly. It can also support a more granular understanding of the acquisition of reasoning capabilities throughout the training process and scaling of language models (@biderman2023pythia).

**Evolve over time**: Economic reasoning evolves over time. For example, the Lucas critique (@lucas1976econometric) was influential in shifting macroeconomic modelling, while the credibility revolution described in @angrist2008mostly was similarly influential in microeconomic work. A historical perspective on the thought about causality going back to the early 18th century is found in @heckman2015causal, and Debreu [-@debreu1984economic] describes the evolution of economic theory up until that point. @lewbel2019identification offers a historical perspective on the issue of identification. For this reason, it is important to consider new works as they are incorporated in the live economic debate. This can be most directly done by drawing from academic papers in general interest economic journals, which benefit from wide impact in the profession. However, there are two main drawbacks of using academic papers to proxy for the development of economic reasoning over time. The first is the widely discussed publication bias (@andrews2019identification), but a perhaps equally important issue is that of unobserved false negatives: if many contributions that are now considered classics have been previously rejected (@mighty1994fallen), there are probably many others who will not be available for the incorporation as a benchmark task.

**Cover different levels of economic reasoning**: An ideal economic reasoning benchmark tests whether the model is able to recognise increasingly sophisticated levels of economic reasoning. When faced with $Q$ that contains a statemet of the economic problem, and a summary of the methodology and main findings, an AI model must recognise the type of analysis that was conducted. Drawing from the definitions more recently stated in @heckman2023econometric, those are, in order of analytical prowess, (a) the impact of a given intervention in a specific environment; (b) understanding the mechanisms by which the intervention might work; (c) forecasting the effects of the same intervention in other environments or states of the world; and (d) forecasting the effects of never-before-implemented interventions in various environments.

**Receive inputs from the public**: In order to truly reflect the breadth and diversity in economic thought, an ideal benchmark should be open to receiving suggested questions from the public. For example, economists publishing a new paper could suggest a source question based on their work. A practical way to achieve this is to create clear instructions and a standardised form that would be filled by that external user presenting the suggestion, coupled with a script that takes in the source quesiton(s) and introduces the necessary variations in information, knowledge and logic to achieve identification.

The benchmark should also result in a metric that is not subject to the false "emerging abilities" results (@schaeffer2024emergent), for example the Brier score (@brier1950verification).

# Preliminary considerations[^concl]

[^concl]: This section is the basis for the conclusions in a future version after the empirical estimation is completed.

The model in this paper resembles the more sophisticated idea by LeCun [-@lecun2022path] that AIs require a combination of "mental" modules that can separably executve perception (eg, take in a prompt that might be text-only or text-and-image), calculate the cost of processing, and imagine action sequences.

 As economic agents and policymakers harness generative artificial intelligence (AI) to reap considerable efficiencies, and thus their societal footprint becomes larger, a benchmark for economic reasoning is needed. I suggest ways to implement such a benchmark, and measure the current performance of a selected list of LMs.

Understanding AI models' ability to reason and go beyond a pure probabilistic exercise is crucial as these models have an increasing importance in society. For example, models that suggest economic actions to people should better reflect latent, structural models that represent people's preferences or well-being rather than simply a prediction based on their observed behaviour, as argued by @kleinberg2023inversion. Models that reason better can rise up to the challenge of learning actual metrics of interest instead of real-world measurements, because the latter have added noise from human cognitive and heuristics limitations, which are then amplified by multiple types of biases in data (CITE paper on multiple biases). And with the increasing linguistic prowess of language models, their use in the economic research process (@korinek2023gen, @ludwig2024machine) is likely to increase, putting a premium on the ability to measure how well these models can reason.

Let me conclude with Ken Arrow's impossibility theorem (CITE), or rather the story of how he achieved this incredibly influential result. Arrow first attempted to improve upon two-century-old Condorcet's paradox, and studied ways in which individual preferences could be aggregated while satisfying some intuitive conditions. It was only through repeated failures to do so that he switched the focus to attempting to prove its impossibility. While Arrow can be safely used as a prime example of economic reasoning, the point this anecdote illustrates is that breakthroughs in economic knowledge require also inspiration (in this case from the appeal of addressing Condorcet's paradox) as well as persistence and ability to change one's focus. The current work focuses on developing robust benchmarks of models' reasoning abilities in economics; further work exploring their contributions to inspiration[^ideation] and to methodological assistance (as in the example to change focus) are also warranted for a more complete assessment of models' abilities to provide cognitive support to human economists.

[^ideation]: @korinek2023gen illustrates use of AI models to help economists have new ideas for work.

# Annex 1: discussion of biases in human surveys and how they could affect LM questionnaires

* Section A-4 in @stantcheva2023run

The goal of this annex is to list side-by-side the main human biases that affect survey responses and their corresponding machine version, if any (from a theoretical perspective - it would be interesting to test if LMs carry over some of these biases that are supposed to be only human, which could suggest they are parroting or in extremis developing sources of bias like shame, etc).

# References